In [2]:
import cv2
import pickle
import numpy as np
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("car_parking_model.h5")

# Load saved rectangles
with open("CarParkPos", "rb") as f:
    rectangles = pickle.load(f)

# Open video
cap = cv2.VideoCapture("carPark.mp4")

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define VideoWriter to save predictions
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # codec
out = cv2.VideoWriter("carPark_output.mp4", fourcc, fps, (width, height))

delay = int(1000 / fps)  
frame_skip = 2            # process every 2nd frame
frame_count = 0
last_frame = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    occupied_count, empty_count = 0, 0

    if frame_count % frame_skip == 0:
        batch_slots, coords = [], []
        for rect in rectangles:
            x1, y1, x2, y2 = rect
            slot = frame[y1:y2, x1:x2]

            if slot.size == 0:
                continue

            slot_resized = cv2.resize(slot, (100, 50)) / 255.0
            batch_slots.append(slot_resized)
            coords.append((x1, y1, x2, y2))

        if batch_slots:
            batch_slots = np.array(batch_slots)
            preds = model.predict(batch_slots, verbose=0).flatten()

            # Zip for pairing 
            for (x1, y1, x2, y2), pred in zip(coords, preds):
                if pred > 0.5:
                    color = (0, 0, 255)  # Red = Occupied
                    occupied_count += 1
                else:
                    color = (0, 255, 0)  # Green = Empty
                    empty_count += 1

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # Draw summary box
        cv2.rectangle(frame, (10, 10), (220, 70), (50, 50, 50), -1)  # background
        cv2.putText(frame, f"Empty: {empty_count}", (20, 35),
                    cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, f"Occupied: {occupied_count}", (20, 60),
                    cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 0, 255), 1, cv2.LINE_AA)

        last_frame = frame.copy()

    # Show last processed frame
    if last_frame is not None:
        cv2.imshow("Parking Detection", last_frame)
        out.write(last_frame)  # save prediction frame
    else:
        cv2.imshow("Parking Detection", frame)
        out.write(frame)  # save raw frame (before first prediction)

    if cv2.waitKey(delay) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("Processing finished & saved as carPark_output.mp4")


Processing finished & saved as carPark_output.mp4
